# Multicollinearity

Multicollinearity is an assumption of regression that explores whether the predictor attributes (X) are correlated. Linear regression assumes that the predictor attributes are uncorrelated, but in practice this assumption is never met. What impacts does multicollinearity have on the linear regression? The short answer is that the regression parameters (the $\beta$ terms) will be more or less unbiased, however, the same cannot be said for the standard errors. 

Let's first use some real data to explore this, then we'll use simulated data to show the impact correlated predictor attributes have on the linear regression estiamtes. 

In [ ]:
library(tidyverse)
library(ggformula)

theme_set(theme_bw(base_size = 18))

college <- read_csv("https://raw.githubusercontent.com/lebebr01/statthink/main/data-raw/College-scorecard-4143.csv") %>%
  mutate(act_mean = actcmmid - mean(actcmmid, na.rm = TRUE),
         cost_mean = costt4_a - mean(costt4_a, na.rm = TRUE)) %>%
  drop_na(act_mean, cost_mean)

adm_mult_reg <- lm(adm_rate ~ actcmmid + costt4_a, data = college)

summary(adm_mult_reg)

In [ ]:
library(mosaic)

cor(actcmmid ~ costt4_a, data = college)

## Variance Inflation Factor

The variance inflation factor (VIF) is a commonly used statistic to aid in diagnosing multicollinearity. It attempts to estimate how much the variance of the estimated regression equation is inflated due to correlated predictor attributes. 

First, this can be calculated from the car package, using the function `vif()`. 

In [ ]:
library(car)

vif(adm_mult_reg)

Fundamentally, the VIF is calculated with the following steps. 

1. Fit a regression where one of the X attributes is the outcome and the remaining X attributes are predictors. 
2. Calculate VIF: $VIF = 1 / 1 - R^2$ from step 1. 
3. Repeate this for all X attributes.
4. Evaluate extent to which VIF is problematic. Rules of thumb include VIF statistics greater than 5 or 10. 

Interpretation wise, the square root of the VIF statistic can provide an indication of how inflated the standard error would be if the predictors are uncorrelated. 

In [ ]:
act_lm <- lm(actcmmid ~ costt4_a, data = college)
summary(act_lm)$r.square

In [ ]:
1 / (1 - .3088)

In [ ]:
sqrt(1.446)

### Simulated data exploration

Let's use some simulated data to show the extent to which multicollinearity can be problematic, particularly for more highly correlated attributes. 

First, let's start with uncorrelated (on average) predictor attributes.

In [ ]:
library(simglm)

sim_args <- list(formula = y ~ 1 + act + gpa + sat, 
                 fixed = list(act = list(var_type = 'continuous',
                                         mean = 20, 
                                         sd = 4),
                              gpa = list(var_type = 'continuous',
                                         mean = 2, 
                                         sd = .5),
                              sat = list(var_type = 'continuous',
                                         mean = 500, 
                                         sd = 100)),
                 correlate = list(fixed = data.frame(x = c('act', 'act', 'gpa'), 
                                                     y = c('gpa', 'sat', 'sat'), 
                                                     corr = c(0, 0, 0))),
                error = list(variance = 100),
                reg_weights = c(1, 1, 1, 1),
                 sample_size = 10000)

sim_data <- simulate_fixed(data = NULL, sim_args) %>%
   simulate_error(sim_args) %>%
   correlate_variables(sim_args) %>%
   generate_response(sim_args)

head(sim_data)

In [ ]:
cor(sim_data[c('y', 'act', 'gpa', 'sat')])

In [ ]:
sim_lm <- lm(y ~ 1 + act + gpa + sat, data = sim_data)
summary(sim_lm)

In [ ]:
vif(sim_lm)

Let's now increase the correlation among the attributes. 

In [ ]:
library(simglm)

sim_args <- list(formula = y ~ 1 + act + gpa + sat, 
                 fixed = list(act = list(var_type = 'continuous',
                                         mean = 20, 
                                         sd = 4),
                              gpa = list(var_type = 'continuous',
                                         mean = 2, 
                                         sd = .5),
                              sat = list(var_type = 'continuous',
                                         mean = 500, 
                                         sd = 100)),
                 correlate = list(fixed = data.frame(x = c('act', 'act', 'gpa'), 
                                                     y = c('gpa', 'sat', 'sat'), 
                                                     corr = c(0.5, 0.5, 0.25))),
                error = list(variance = 100),
                reg_weights = c(1, 1, 1, 1),
                 sample_size = 10000)

sim_data <- simulate_fixed(data = NULL, sim_args) %>%
   simulate_error(sim_args) %>%
   correlate_variables(sim_args) %>%
   generate_response(sim_args)

head(sim_data)

In [ ]:
cor(sim_data[c('y', 'act', 'gpa', 'sat')])

In [ ]:
sim_lm2 <- lm(y ~ 1 + act + gpa + sat, data = sim_data)
vif(sim_lm2)

In [ ]:
broom::tidy(sim_lm)
broom::tidy(sim_lm2)

In [ ]:
library(simglm)

sim_args <- list(formula = y ~ 1 + act + gpa + sat, 
                 fixed = list(act = list(var_type = 'continuous',
                                         mean = 20, 
                                         sd = 4),
                              gpa = list(var_type = 'continuous',
                                         mean = 2, 
                                         sd = .5),
                              sat = list(var_type = 'continuous',
                                         mean = 500, 
                                         sd = 100)),
                 correlate = list(fixed = data.frame(x = c('act', 'act', 'gpa'), 
                                                     y = c('gpa', 'sat', 'sat'), 
                                                     corr = c(0.5, 0.9, 0.5))),
                error = list(variance = 100),
                reg_weights = c(1, 1, 1, 1),
                 sample_size = 10000)

sim_data <- simulate_fixed(data = NULL, sim_args) %>%
   simulate_error(sim_args) %>%
   correlate_variables(sim_args) %>%
   generate_response(sim_args)

head(sim_data)

In [ ]:
cor(sim_data[c('y', 'act', 'gpa', 'sat')])

In [ ]:
sim_lm3 <- lm(y ~ 1 + act + gpa + sat, data = sim_data)
vif(sim_lm3)

In [ ]:
broom::tidy(sim_lm)
broom::tidy(sim_lm2)
broom::tidy(sim_lm3)

In [ ]:
library(simglm)

sim_args <- list(formula = y ~ 1 + act + gpa + sat, 
                 fixed = list(act = list(var_type = 'continuous',
                                         mean = 20, 
                                         sd = 4),
                              gpa = list(var_type = 'continuous',
                                         mean = 2, 
                                         sd = .5),
                              sat = list(var_type = 'continuous',
                                         mean = 500, 
                                         sd = 100)),
                 correlate = list(fixed = data.frame(x = c('act', 'act', 'gpa'), 
                                                     y = c('gpa', 'sat', 'sat'), 
                                                     corr = c(0.9, 0.98, 0.85))),
                error = list(variance = 100),
                reg_weights = c(1, 1, 1, 1),
                 sample_size = 10000)

sim_data <- simulate_fixed(data = NULL, sim_args) %>%
   simulate_error(sim_args) %>%
   correlate_variables(sim_args) %>%
   generate_response(sim_args)

head(sim_data)

In [ ]:
cor(sim_data[c('y', 'act', 'gpa', 'sat')])

In [ ]:
sim_lm4 <- lm(y ~ 1 + act + gpa + sat, data = sim_data)
vif(sim_lm4)

In [ ]:
broom::tidy(sim_lm)
broom::tidy(sim_lm2)
broom::tidy(sim_lm3)
broom::tidy(sim_lm4)